I started with collecting the data from newsapi.org

In [3]:
import requests
apikey='72bbb827e6a1454f8c2b3832e6ca33d9'

# I decided to look at these 20 domains (all USA journals)
twenty_domains=[
   'abcnews.go.com',
    'apnews.com','bloomberg.com', 'breitbart.com',
    'cbsnews.com',
                'cnbc.com', 'cnn.com', 'foxnews.com', 'msnbc.com','nbcnews.com',
                'politico.com', 'theamericanconservative.com', 'thehill.com', 'huffingtonpost.com', 
                'washingtonpost.com',  'time.com', 'usatoday.com/news', 'wsj.com','nymag.com']
# the key words (queries) passed to the api to return articles that contains them 
queries=['Joe OR Biden','Bernie OR Sanders','Elizabeth OR warren', 'pete OR buttigieg','michael OR bloomberg']
list_of_lists=[]
c=1
#page limit set to 6 so range (1,6) can only request pages 1,2,3,4,5 which are allowed in the free version
limit=6
domainorder=0
for q in queries:

    mylist=[]
    
    for dm in twenty_domains:

        c=1
        limit=5
        for page in range (1,6):

            if (c>=limit):
                break;
            #the date was changed and collected for multiple date ranges however at any point in time
            # the free version of this api allows us to request articles that are no more than 1 month old
            url1 = ('http://newsapi.org/v2/everything?'
               'q='+q+'&domains='+dm+
                '&from=2020-03-27&to=2020-03-27&'
               'sortBy=popularity&page='+str(page)+
               '&apiKey='+apikey)

            response = requests.get(url1)
            mylist.append(response.json())
            
            #The Api free version allows only for 500 requests per day so in order to minimize unnecessary requests
            # I need to decide on the number of pages requested, so if the total results returned from a particular domain are less than 20 or 40
            # then I don't need to request multiple pages as they will be returned empty. Since every page contains 20 results 
            # therefore from the total result per source I could decide on the number of pages (limit variable) I need to request

            if (mylist[0]['totalResults']<81):
                limit=5
            if (mylist[0]['totalResults']<61):
                limit=4
            if (mylist[0]['totalResults']<41):
                limit=3
            if (mylist[0]['totalResults']<21):
                limit=2
            if (mylist[0]['totalResults']==0):
                limit=1

            c=c+1

    list_of_lists.append(mylist)
    domainorder=domainorder+1

In [4]:
list_of_lists[0]

[{'status': 'ok', 'totalResults': 0, 'articles': []},
 {'status': 'ok',
  'totalResults': 4,
  'articles': [{'source': {'id': 'associated-press',
     'name': 'Associated Press'},
    'author': '',
    'title': 'Guest lineups for the Sunday news shows',
    'description': 'WASHINGTON (AP) — Guest lineups for the Sunday news shows:',
    'url': 'https://apnews.com/beafbe1d484050ac97438c7bd6a248e4',
    'urlToImage': 'https://bloximages.newyork1.vip.townnews.com/tulsaworld.com/content/tncms/custom/image/aacc4e10-0064-11e7-adc2-d3321407a1f8.jpg',
    'publishedAt': '2020-03-27T22:56:58Z',
    'content': 'WASHINGTON (AP) Guest lineups for the Sunday news shows:\r\nABC\'s This Week" White House economic adviser Larry Kudlow; Gov. John Bel Edwards, D-La.; former White House homeland security adviser Tom Bossert.\r\nNBC\'s Meet the Press Edwards; Democratic presidentia… [+911 chars]'},
   {'source': {'id': 'associated-press', 'name': 'Associated Press'},
    'author': 'By KRISTIN M. HALL AP E

All the json responses collected were saved in list_of_lists
Now we want to seperate the dictionaries values and save them in different lists. One for the sources, and one for the respective titles,
contents, date, authors and urls.
For every url we initiate a an http request to scrape the article text using Beautiful soup

In [5]:
from newspaper import fulltext
from bs4 import BeautifulSoup



sources=[]
titles=[]
contents=[]
authors=[]
dates=[]
urls=[]
texts=[]
mlist=[]

resnumber=[]

c=0
for lis in range(0,len(list_of_lists)):

    mlist=list_of_lists[lis]

    for domain in range(0,len(mlist)):

        if ((mlist[domain]['status']=='ok')==False):
            continue
        data=mlist[domain]['articles']
        if (not data):
            continue
        for k in range(0,len(data)):
            notadded=True
            sources.append(data[k]['source']['name'])
            titles.append(data[k]['title'])
            dates.append(data[k]['publishedAt'])
            contents.append(data[k]['content'])
            authors.append(data[k]['author'])
            url=data[k]['url']
            urls.append(url)
            resnumber.append(c)
            c=c+1
            html = requests.get(url).text
            soup = BeautifulSoup(html, 'lxml')
            try:

                text = fulltext(html)
                texts.append(text)

             
                notadded=False
            
            except:
                   

                if (notadded):
                    texts.append(soup.text)

                    

Saving the arranged data in a dataframe and a csv file:

In [65]:
import pandas as pd
from datetime import datetime
import csv
# Converting datetime object to string
dateTimeObj = datetime.now()
 
timestampStr = dateTimeObj.strftime("%d%b%Y_%H%M%S%f")

newdf=pd.DataFrame(list(zip(resnumber,sources,titles,contents,dates,authors,urls, texts)),
              columns=['resnumber','sources','titles','contents','date','authors','urls','texts'])
newdf.to_csv('newsdata_'+timestampStr+'.csv',header=True)

Cleaning the collected data:

In [71]:
toberemoved=['photo','getty','images', 'advertisement','"','jeff yang','nayyera haq', 'cnn ','(cnn)', ' cbs ', 'fox news',
            "chat with us in facebook messenger. find out what's happening in the world as it unfolds.",' on air videos'
             'abcnews', 'apnews','bloomberg.com', 'breitbart','cbsnews','cnbc', 'msnbc','nbcnews','politico', 
             'theamericanconservative', 'thehill', 'huffingtonpost', 'washingtonpost',  'time.com', 'usatoday',
             'Click to copy\n','\nClick to copy',  
            '403 Forbidden\n','\n403 Forbidden','\nnginx','get all the latest news on coronavirus and more delivered daily to your inbox. sign up here',
            
             'Why did this happen?','\n','Please make sure your browser supports JavaScript and cookies and that you are not blocking them from loading. For more information you can review our Terms of Service and Cookie Policy.',
'live blog / live coronavirus updates: u.s. and global news on covid-19  here are the latest updates from around the world.'
            ]

def cleantext_function(sentence):
    
    sentence=sentence.lower()

    sentence=sentence.replace("read more",'')
   
    for n in toberemoved:
        sentence = sentence.replace(n.lower(),' ')
    # split sentence into words
    #listofwords = sentence.split(' ')
    error="If you typed the URL into your browser, check that you entered it correctly. If you reached this page via our site or search, please let us know by emailing support@wsj.com"
    if ( sentence.find(error.lower())>=0):
            sentence=' '
    sentence=sentence.replace("n\'t",' not')
    

    return sentence

In [72]:
newdf.set_index('resnumber')
cleanedtext=[]
for t in range(0, len(texts)):
    changed_text=cleantext_function(texts[t])
    if (changed_text.strip()!=''):
        cleanedtext.append(changed_text)
    else:
        newdf.drop(resnumber[t], inplace=True)

In [100]:
dateTimeObj = datetime.now()
 
timestampStr = dateTimeObj.strftime("%d%b%Y_%H%M%S%f")
import pandas as pd
newcleaned=newdf.copy()
#newcleaned.set_index('resnumber')
newcleaned['cleaned_text']=cleanedtext

The data for different date ranges was concatenated using concat function:
   alldf = pd.concat([newcleaned, newcleaned_1], axis=0, ignore_index=True)

We then removed duplicate rows if they exist:

In [101]:
newcleaned.drop('resnumber', axis=1, inplace=True)
newcleaned.drop_duplicates(inplace=True)
newcleaned

,sources,titles,contents,date,authors,urls,texts,cleaned_text
0,Associated Press,Guest lineups for the Sunday news shows,WASHINGTON (AP) Guest lineups for the Sunday n...,2020-03-27T22:56:58Z,,https://apnews.com/beafbe1d484050ac97438c7bd6a...,Guest lineups for the Sunday news shows\n\nWAS...,guest lineups for the sunday news shows washi...
1,Associated Press,Country singer Joe Diffie tests positive for c...,"NASHVILLE, Tenn. (AP) A publicist for Joe Diff...",2020-03-27T19:33:49Z,By KRISTIN M. HALL AP Entertainment Writer,https://apnews.com/3e2edb7ce690346df645d79dae4...,"FILE - This Aug. 22, 2018 file photo shows Joe...","file - this aug. 22, 2018 file shows joe dif..."
2,Associated Press,Milwaukee's black community hit hard by corona...,"MADISON, Wis. (AP) Deaths and confirmed cases ...",2020-03-27T17:22:54Z,By SCOTT BAUER Associated Press,https://apnews.com/b52e4e9a63d64e3a25109f09010...,The onramp to the Hoan Bridge is void of traff...,the onramp to the hoan bridge is void of traff...
3,Associated Press,"Column: As we battle virus, don't forget 1919 ...","After a week of travel, which included a coupl...",2020-03-27T21:47:11Z,PAUL NEWBERRY AP Sports Columnist,https://apnews.com/6f1fef1bcf19f9b0eb9cd8bce39...,"FILE - In this Feb. 4, 2005, file photo, the i...","file - in this feb. 4, 2005, file , the inscr..."
5,Breitbart News,"Joe Rogan Worried by 'Old,' 'Stumbling' Biden:...",Podcast host Joe Rogan blasted the Democrat Pa...,2020-03-27T15:29:30Z,"Hannah Bleau, Hannah Bleau",https://www.breitbart.com/politics/2020/03/27/...,Podcast host Joe Rogan blasted the Democrat Pa...,podcast host joe rogan blasted the democrat pa...
...,...,...,...,...,...,...,...,...
311,Time,President Trump and China’s Xi Pledge to Coope...,U.S. President Donald Trump and China’s Xi Jin...,2020-03-27T10:10:35Z,Karen Leigh / Bloomberg,https://time.com/5811238/trump-xi-cooperate-co...,U.S. President Donald Trump and China’s Xi Jin...,u.s. president donald trump and china’s xi jin...
312,Time,Report of Urns Stacked at Wuhan Funeral Homes ...,(Bloomberg) — The long lines and stacks of ash...,2020-03-27T08:05:28Z,Bloomberg,https://time.com/5811222/wuhan-coronavirus-dea...,(Bloomberg) — The long lines and stacks of ash...,(bloomberg) — the long lines and stacks of ash...
313,Time,Trump’s Iran Gamble: Will “Maximum Pressure” A...,"On a recent evening in Tehran, apartment dwell...",2020-03-27T21:19:21Z,Kimberly Dozier and John Walcott,https://time.com/5811772/trumps-iran-gamble/,"On a recent evening in Tehran, apartment dwell...","on a recent evening in tehran, apartment dwell..."
316,New York Magazine,How Will New York’s Rent Freeze Work?,Were committed to keeping our readers informed...,2020-03-27T22:34:57Z,James D. Walsh,http://nymag.com/intelligencer/2020/03/how-wil...,Photo: Craig Warga/Bloomberg via Getty Images\...,: craig warga/bloomberg via we’re commit...


Saving the cleaned data into a csv file:

In [102]:
newcleaned.to_csv('cleaned.csv')